The goal of this notebook is to share how to access LINDA data from python with a minimal instalation from within the confederation

### Using only requests

In [13]:
#!python -m pip install requests pandas ipykernel

In [14]:
import pandas as pd
import requests
import json

In [8]:
endppoint_ch = 'http://int.lindas.admin.ch/query'
header = {"Accept": "application/sparql-results+json"} 

proxies = {
	"http": "your_proxy",
	"https": "your_proxy"
}  

In [15]:
query = """
	PREFIX station: <https://environment.ld.admin.ch/foen/hydro>
	SELECT *
	FROM <https://lindas.admin.ch/foen/hydro>
	WHERE { 
		?a ?b ?c.
		filter contains(str(?a), "observation")
	} LIMIT 3
"""

In [ ]:
session = requests.Session()
session.proxies.update(proxies)
session.proxies.values()

In [ ]:
r = session.get(url = endppoint_ch, 
				headers = header, 
				params= {'format': 'json', 'query': "".join(query)},
				verify = False)
data = r.json()


In [18]:
data

{'head': {'vars': ['a', 'b', 'c']},
 'results': {'bindings': [{'a': {'type': 'uri',
     'value': 'https://environment.ld.admin.ch/foen/hydro/river/observation/2232'},
    'b': {'type': 'uri', 'value': 'http://example.com/isLiter'},
    'c': {'type': 'literal', 'value': 'true'}},
   {'a': {'type': 'uri',
     'value': 'https://environment.ld.admin.ch/foen/hydro/river/observation/2629'},
    'b': {'type': 'uri', 'value': 'http://example.com/isLiter'},
    'c': {'type': 'literal', 'value': 'true'}},
   {'a': {'type': 'uri',
     'value': 'https://environment.ld.admin.ch/foen/hydro/river/observation/2203'},
    'b': {'type': 'uri', 'value': 'http://example.com/isLiter'},
    'c': {'type': 'literal', 'value': 'true'}}]}}

### Using graphly
zazuko provides a library for this too..

In [23]:
# !git clone https://github.com/zazuko/graphly
##  WARNING: you will need to add to each .get in graphly.api_client.py "verify = False" before installing, if you are inside the confederation
# !python -m pip install -e  graphly

In [24]:
from graphly.api_client import SparqlClient

In [25]:
sparql = SparqlClient("https://environment.ld.admin.ch/query")
sparql.session.proxies.update({"http": "your_proxy", "https":"your_proxy"})
sparql.add_prefixes({
    "rdf": "<http://www.w3.org/1999/02/22-rdf-syntax-ns#>",
    "rdfs": "<http://www.w3.org/2000/01/rdf-schema#>"
})

In [ ]:
query = """
SELECT * WHERE {
  ?sub ?pred ?obj .
} LIMIT 10
"""

df = sparql.send_query(query)
df.head()